In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm import tqdm
from Base.DataIO import DataIO

#----Recommenders----
from SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from SLIM.SlimElasticNet import SLIMElasticNetRecommender
from cf.item_cf3 import ItemBasedCollaborativeFiltering
from cf.user_cf2 import UserBasedCollaborativeFiltering
from MF.IALSRecommender import IALSRecommender
from cbf.cbf import ContentBasedFiltering
from SlimBPR.SlimBPRRec import SlimBPRRec
from SlimBPR.SlimBPR import SlimBPR
from MF.ALS import AlternatingLeastSquare


from Hybrid.hybridRec import HybridRecommender
#---------------

#----Model Load & Save----
from Data_manager.Dataset import Dataset
#-------------------------

from sklearn.model_selection import train_test_split

In [ ]:
def load_data_ICM():
  return pd.read_csv("./data_ICM_title_abstract.csv")

In [ ]:
features=load_data_ICM()
d ={'item_id': features['row'],'feature_id':features['col'],'value':features['data']}
features=pd.DataFrame(data=d)
itemList=list(d['item_id'])

In [ ]:
featureList=list(d['feature_id'])

In [ ]:
valueList=list(d['value'])
ICM = sp.coo_matrix((valueList,(itemList,featureList)))
ICM = ICM.tocsr()

In [ ]:
ICM

**Dataset splitting into train,validation and test**

This is the last part before creating the recommender. However, this step is super important, as it is the base for the training, parameters optimization, and evaluation of the recommender(s).

In here we read the ratings (which we loaded and preprocessed before) and create the train, validation, and test User-Rating Matrices (URM). It's important that these are disjoint to avoid information leakage from the train into the validation/test set, in our case, we are safe to use the train_test_split function from scikit-learn as the dataset only contains one datapoint for every (user,item) pair. On another topic, we first create the test set and then we create the validation by splitting again the train set.

train_test_split takes an array (or several arrays) and divides it into train and test according to a given size (in our case testing_percentage and validation_percentage, which need to be a float between 0 and 1).

After we have our different splits, we create the sparse URMs by using the csr_matrix function from scipy.




In [ ]:
#Load
loaded_dataset = Dataset(None, None, None, None)
loaded_dataset.load_data("Salvataggi/")


urm_train = (loaded_dataset.get_urm_train())["urm_train"]
urm_validation = (loaded_dataset.get_urm_validation())["urm_validation"]
urm_test = loaded_dataset.get_urm_test()["urm_test"]

In [ ]:
urm_train_validation = urm_train + urm_validation

**Evaluation Metrics**

In [ ]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

**Evaluation Procedure**

The evaluation procedure returns the averaged accuracy scores (in terms of precision, recall and MAP) for all users (that have at least 1 rating in the test set). It also calculates the number of evaluated and skipped users. It receives a recommender instance, and the train and test URMs.

In [ ]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
            
#         recommendations = recommender.recommend(user_id_array=user_id,
#                                                cutoff=recommendation_length,
#                                                remove_seen_flag=True
#                                                )

        expected_ratings = recommender.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                        assume_unique=True, invert=True)

        recommendations = recommended_items[unseen_items_mask]


        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1

    
    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)
    
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

**Tuning ALS**

In [ ]:
import time

best_factors = np.zeros(4000)
best_reg = np.zeros(4000)
best_MAP = np.zeros(4000)

counter = 0
start = time.time()
for factors in range(100, 1000, 40) :
    
    reg = 1.0
    for i in range(0, 20) :
        
        print("Iteration " + str(counter) + " factors: " + str(factors)   + " reg: " + str(reg)  + " started.")
        
        recommenderToTune = AlternatingLeastSquare(urm_train) #factors 0-1000, reg 0.15-2, iterations:30
        recommenderToTune.fit(factors, reg, 30)

        accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                                urm_train_validation, 
                                                                                                urm_test)
        
        print("MAP iteration " + str(counter) + ": " + str(accum_map))
        best_factors[counter] = factors
        best_reg[counter] = reg
        best_MAP[counter] = accum_map
        
        counter += 1
        reg += 0.05
        
stop = time.time()
print("Execution Time: " + str(stop-start))

In [ ]:
best_MAP

In [ ]:
#Tieni solo i migliori 400
factors_top400 = np.zeros(400, dtype=int)
reg_top400 = np.zeros(400, dtype=float)
MAP_top400 = np.zeros(400)

#Sort of best maps decrescent
MAP_sorted = np.sort(best_MAP)[::-1]

#Tengo solo i primi 400 MAP
MAP_top400 = MAP_sorted[:400]

counter = 0

#MAP_top400[0] will be the highest mapping_value 
for mapping_value in MAP_top400 :
    
    for iteration in range(0, best_MAP.shape[0]) :
        
        if best_MAP[iteration] == mapping_value :
            
            factors_top400[counter] = best_factors[iteration]
            reg_top400[counter] = best_reg[iteration]
            
            counter += 1
            
print(factors_top400)
print(reg_top400)
print(MAP_top400)

In [ ]:
print("Inizio tuning preciso di +20 -20 factors dei top400")

iteration = 0
for iteration in tqdm(range(0, factors_top400.shape[0])) :
        
    start = time.time()
    print("Iteration " + str(iteration) + " factors: " + str(factors_top400[iteration])   + " Reg: " + str(reg_top400[iteration])  + " started.")
    
    print("factors +20")
    print("factors: " + str(factors_top400[iteration] + 20) + " reg: " + str(reg_top400[iteration]))
    
    recommenderToTune.fit(factors_top400[iteration]+20, reg_top400[iteration], 30)

    accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

    print("MAP iteration " + str(iteration) + ": " + str(accum_map))

    if accum_map > MAP_top400[iteration] :
        
        factors_top400[iteration] = factors_top400[iteration] + 20
        reg_top400[iteration] = reg_top400[iteration]
        MAP_top400[iteration] = accum_map
        
        print("updated! factors +20 is better")
        
    ###############
    
    print("factors +10")
    print("factors: " + str(factors_top400[iteration] + 10) + " reg: " + str(reg_top400[iteration]))
    
    recommenderToTune.fit(factors_top400[iteration]+10, reg_top400[iteration], 30)

    accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

    print("MAP iteration " + str(iteration) + ": " + str(accum_map))

    if accum_map > MAP_top400[iteration] :
        
        factors_top400[iteration] = factors_top400[iteration] + 10
        reg_top400[iteration] = reg_top400[iteration]
        MAP_top400[iteration] = accum_map
        
        print("updated! factors +10 is better")
        
    ###############
    
    
    if (factors-20) > 0 :
        print("factors -20")
        print("factors: " + str(factors_top400[iteration] -20) + " reg: " + str(reg_top400[iteration]))

        recommenderToTune.fit(factors_top400[iteration]-20, reg_top400[iteration], 30)

        accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                                urm_train_validation, 
                                                                                                urm_test)

        print("MAP iteration " + str(iteration) + ": " + str(accum_map))

        if accum_map > MAP_top400[iteration] :

            factors_top400[iteration] = factors_top400[iteration] -20
            reg_top400[iteration] = reg_top400[iteration]
            MAP_top400[iteration] = accum_map

            print("updated! factors -20 is better")
        
    ###############
    
    if (factors-10) > 0 :
        print("factors -10")
        print("factors: " + str(factors_top400[iteration] -10) + " reg: " + str(reg_top400[iteration]))

        recommenderToTune.fit(factors_top400[iteration]-10, reg_top400[iteration], 30)

        accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                                urm_train_validation, 
                                                                                                urm_test)

        print("MAP iteration " + str(iteration) + ": " + str(accum_map))

        if accum_map > MAP_top400[iteration] :

            factors_top400[iteration] = factors_top400[iteration] -10
            reg_top400[iteration] = reg_top400[iteration]
            MAP_top400[iteration] = accum_map

            print("updated! factors -10 is better")
        
    ###############

    stop = time.time()
    print("Execution Time: " + str(stop-start))

In [ ]:
#Prendo i 5 migliori
factors_top5 = np.zeros(400, dtype=int)
reg_top5 = np.zeros(400, dtype=float)
MAP_top5 = np.zeros(5)

#Sort of best maps decrescent
MAP_sorted = np.sort(best_MAP)[::-1]

#Tengo solo i primi 400 MAP
MAP_top5 = MAP_sorted[:5]

counter = 0

#MAP_top400[0] will be the highest mapping_value 
for mapping_value in MAP_top5 :
    
    for iteration in range(0, MAP_top400.shape[0]) :
        
        if best_MAP[iteration] == mapping_value :
            
            factors_top5[counter] = factors_top400[iteration]
            reg_top5[counter] = reg_top400[iteration]
            
            counter += 1
            
print(factors_top5)
print(reg_top5)
print(MAP_top5)

In [ ]:
#in un intorno di 10 prendo il migliore per ognuno

for iteration in range (0, MAP_top5.shape[0]) :
    
    for factors in range ((factors_top5[iteration]-5), (factors_top5[iteration]+5)) :
        
        reg = reg_top5[iteration] - 0.05
        for i in range (0, 10) :
            
            print("factors: " + str(factors) + " reg: " + str(reg))

            recommenderToTune.fit(factors, reg, 30)

            accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                                    urm_train_validation, 
                                                                                                    urm_test)

            print("MAP " + str(accum_map))

            if accum_map > MAP_top5[iteration] :

                factors_top5[iteration] = factors
                reg[iteration] = reg
                MAP_top5[iteration] = accum_map

                print("updated! is better")
                  
            reg+= 0.01
            
print(factors_top5)
print(reg_top5)
print(MAP_top5)

In [ ]:
#best values
bestFactors = 0
bestReg = 0
bestMap = 0

#Sort of best maps decrescent
MAP_sorted = np.sort(MAP_top5)[::-1]

#Tengo solo i primi 400 MAP
bestMap = MAP_sorted[0]

counter = 0

for iteration in range(0, MAP_top5.shape[0]) :

    if MAP_top5[iteration] == bestMap :

        bestFactors = factors_top5[iteration]
        bestReg = reg_top5[iteration]

            
print(bestFactors)
print(bestReg)
print(bestMap)

In [ ]:
result = "bestKnn: " + str(bestKnn) + " bestShrink: " + str(bestShrink) + " best MAP: " + str(bestMap)
with open('resultALS.txt', 'a') as fp:
    fp.write(result)

**Recommenders Parameters**

In [ ]:
#Params for the recommenders
cbf_param = {
    "knn": 380,
    "shrink": 9
}

user_cf_param = {
    "knn": 249,
    "shrink": 853
}

item_cf_param = {
    "knn": 119,
    "shrink": 430
}

slim_bpr_param = {   #work in progres....
    "learning_rate" : 0.00214168231523243,
    "epochs": 10,
    "nnz" : 0.39651777555716483,
    "knn": 1134
}

als_param = {   #work in progress...
    "n_factors": 242,
    "regularization": 2.0,
    "iterations": 15
}

Ials_param = {
    "num_factors" : 50,
    "confidence_scaling" : "linear",
    "alpha" : 0.40,
    "epsilon" : 0.28,
    "reg" : 0.0003
}


In [ ]:
userCF = UserBasedCollaborativeFiltering(urm_train)

In [ ]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
itemCF = ItemBasedCollaborativeFiltering(urm_train)

In [ ]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
SlimBpr=SlimBPRRec(urm_train)

In [ ]:
SlimBpr.fit(nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
cbf = ContentBasedFiltering(urm_train,ICM)

In [ ]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
ALS = AlternatingLeastSquare(urm_train)

In [ ]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
IALS = IALSRecommender(urm_train)

In [ ]:
IALS.fit(num_factors=Ials_param["num_factors"], confidence_scaling=Ials_param["confidence_scaling"], alpha=Ials_param["alpha"], epsilon=Ials_param["epsilon"], reg=Ials_param["reg"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(IALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Hybrid**

**Hybrid Recommender Tuning**

In [ ]:
best_weight = 0
best_MAP = 0.0
counter = 0

In [ ]:
import time

#Create recommender
rec = HybridRecommender(urm_train_validation, ICM, userCF, itemCF, cbf, ALS, SlimBpr)


for w_bpr in range(0,1000) :

    start = time.time()
    print("Iteration " + str(counter) + " Weight: " + str(w_bpr)  + " started.")
    
    rec.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
            slim_param=slim_bpr_param,als_param=als_param, w_user=37, w_item=235, w_cbf=98, w_als=73, w_slim_bpr=w_bpr)

    accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(rec, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

    if accum_map > best_MAP :
        best_weight = w_bpr
        print("New Best! weight: " + str(w_bpr) + " MAP: " + str(accum_map) + " previous best MAP: " + str(best_MAP))
        best_MAP = accum_map

    stop = time.time()
    print("Execution Time: " + str(stop-start))

In [ ]:
w = {
    "user_cf": 37,
    "item_cf": 235,
    "cbf": 98,
    "icm_svd": 0,
    "als": ,
    "slim_bpr": 113,
    "elastic": 0
}

In [ ]:
userCF = UserBasedCollaborativeFiltering(urm_train_validation)

In [ ]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
itemCF = ItemBasedCollaborativeFiltering(urm_train_validation)

In [ ]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
SlimBpr=SlimBPRRec(urm_train_validation)

In [ ]:
SlimBpr.fit(nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
cbf = ContentBasedFiltering(urm_train_validation,ICM)

In [ ]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
ALS = AlternatingLeastSquare(urm_train_validation)

In [ ]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
recommender = HybridRecommender(urm_train_validation, ICM, userCF, itemCF, cbf, ALS, SlimBpr)

In [ ]:
recommender.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
                slim_param=slim_bpr_param,als_param=als_param, w_user=w["user_cf"], w_item=w["item_cf"], w_cbf=w["cbf"],w_als=w["als"], w_slim_bpr=w["slim_bpr"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Submission to competition**

This step serves as a similar step that you will perform when preparing a submission to the competition. Specially after you have chosen and trained your recommender.

For this step the best suggestion is to select the most-performing configuration obtained in the hyperparameter tuning step and to train the recommender using both the train and validation set. Remember that in the competition you do not have access to the test set.

Another consideration is that, due to easier and faster calculations, we replaced the user/item identifiers with new ones in the preprocessing step. For the competition, you are required to generate recommendations using the dataset's original identifiers. Due to this, this step also reverts back the newer identifiers with the ones originally found in the dataset.

Last, this step creates a function that writes the recommendations for each user in the same file in a tabular format following this format:

csv
<user_id>,<item_id_1> <item_id_2> <item_id_3> <item_id_4> <item_id_5> <item_id_6> <item_id_7> <item_id_8> <item_id_9> <item_id_10>
Always verify the competitions' submission file model as it might vary from the one we presented here.

In [ ]:
def load_goodguys():
  return pd.read_csv("./data_target_users_test.csv")
goodguys=load_goodguys()

In [ ]:
goodguys

In [ ]:
users_to_recommend = np.random.choice(goodguys.user_id,size=goodguys.size, replace=False)
users_to_recommend

In [ ]:
def prepare_submission(users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    
    recommendation_length = 10
    submission = []
    
    for user_id in users_to_recommend :

        recommendations = recommender.recommend2(user_id, urm_train, recommendation_length)

        
        submission.append((user_id, [item_id for item_id in recommendations]))
   
    return submission

In [ ]:
submission = prepare_submission(users_to_recommend, urm_train_validation, recommender)

In [ ]:
submission

In [ ]:
import os
from datetime import datetime

csv_fname = './submission'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

def write_submission(submissions):
    with open(csv_fname, "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [ ]:
write_submission(submission)